# Building a model 

There are many ways to model a time series in order to make predictions. The most popular ways include:

- Moving average.
- Exponential smoothing.
- Double exponential smoothing.
- Triple exponential smoothing.
- Seasonal autoregressive integrated moving average (SARIMA.)

### Moving average
- in general is used to see trends of the data.
- the next oversvation is the mean of all past observations. 
- By changing the value of the window size you can smooth the data and maybe observe some trends. 

In [ ]:
# Moving Average
window_size = 24
data['EnergyConsumption_smoothed'] = data['EnergyConsumption'].rolling(window=window_size).mean()

# Plotting
plt.figure(figsize=(20, 6))

plt.plot(data["Timestamp"], data['EnergyConsumption'], label='Original EnergyConsumption', color='blue')
plt.plot(data["Timestamp"], data['EnergyConsumption_smoothed'], label=f'Moving Average (Window={window_size})', linestyle='--', color='red')

plt.xlabel('Timestamp')
plt.ylabel('EnergyConsumption')
plt.title('Original vs Moving Average')
plt.legend()
plt.show()

### Exponential smoothing
- here applies the similar logic as in the moving average, except less importance is given to the next data as we move further from the present. 
- math. equation: y = AXt + (1-A)Yt-1, t > 0
    - A is a smoothing factor between 0 and 1; it determines how fast the weight decreases for the previous observations. 
    - the closer is A to zero the smoother the line will be and will approach the average. 

### Double exponential smoothing
- used when there is a trend in the time series
- exponential smoothing is used twice: 
    - math. equations: 
    y = AXt + (1-A)(Yt-1 + Bt-1)
    Bt = ß(Yt -Yt-1) + (1-ß)Bt-1
    ß: trend smoothing factor, takes value between 0 and 1
    

### Triple exponential smoothing

- This method extedns the double exponential smoothing by adding a seasonal exponential factor. 
- It is useful when we see seasonality in time series. 
- y(gamma) is the seasonal smoothing factor and L is the length of the season.

### Seasonal autoregressive Integrated Moving average Model (SARIMA)



In [3]:
# ARMA

#follow along the tutorial https://builtin.com/data-science/time-series-model

- ARMA model can explain the relationship of time series with both random noise (moving average) and itself at a previous step (autogregression)

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima_process import ArmaProcess

import matplotlib.pyplot as plt
import numpy as np

import warnings
warnings.filterwarnings("ignore")

%matplotlib inline


In [ ]:
plt.rcParams["figure.figsize"]  = [10,7.5]

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

from sklearn.metrics import r2_score, median_absolute_error, mean_absolute_error
from sklearn.metrics import median_absolute_error, mean_squared_error, mean_squared_log_error

from scipy.optimize import minimize
import statsmodels.tsa.api as smt
import statsmodels.api as sm

from tqdm import tqdm

from itertools import product

def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

- mean average percentage error (MAPE) should be calculated. 

In [ ]:
data_date_indexed.head()

In [ ]:
def plot_moving_average(series, window, plot_intervals=False, scale=1.96):

    rolling_mean = series.rolling(window=window).mean()
    
    plt.figure(figsize=(17,8))
    plt.title('Moving average\n window size = {}'.format(window))
    plt.plot(rolling_mean, 'g', label='Rolling mean trend')
    
    #Plot confidence intervals for smoothed values
    if plot_intervals:
        mae = mean_absolute_error(series[window:], rolling_mean[window:])
        deviation = np.std(series[window:] - rolling_mean[window:])
        lower_bound = rolling_mean - (mae + scale * deviation)
        upper_bound = rolling_mean + (mae + scale * deviation)
        plt.plot(upper_bound, 'r--', label='Upper bound / Lower bound')
        plt.plot(lower_bound, 'r--')
            
    plt.plot(series[window:], label='Actual values')
    plt.legend(loc='best')
    plt.grid(True)
    
#Smooth by the previous 5 days (by week)
plot_moving_average(data_date_indexed.EnergyConsumption, 1)

#Smooth by the previous month (30 days)
plot_moving_average(data_date_indexed.EnergyConsumption, 7)

#Smooth by previous quarter (90 days)
plot_moving_average(data_date_indexed.EnergyConsumption, 30, plot_intervals=True)

In [6]:
### Exponential smoothing


def exponential_smoothing(series, alpha):

    result = [series[0]] # first value is same as series
    for n in range(1, len(series)):
        result.append(alpha * series[n] + (1 - alpha) * result[n-1])
    return result
  
def plot_exponential_smoothing(series, alphas):
 
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        plt.plot(exponential_smoothing(series, alpha), label="Alpha {}".format(alpha))
    plt.plot(series.values, "c", label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Exponential Smoothing")
    plt.grid(True);

plot_exponential_smoothing(data_date_indexed.EnergyConsumption, [0.05, 0.3])

In [ ]:
### Double Exponetial smoothing

def double_exponential_smoothing(series, alpha, beta):

    result = [series[0]]
    for n in range(1, len(series)+1):
        if n == 1:
            level, trend = series[0], series[1] - series[0]
        if n >= len(series): # forecasting
            value = result[-1]
        else:
            value = series[n]
        last_level, level = level, alpha * value + (1 - alpha) * (level + trend)
        trend = beta * (level - last_level) + (1 - beta) * trend
        result.append(level + trend)
    return result

def plot_double_exponential_smoothing(series, alphas, betas):
     
    plt.figure(figsize=(17, 8))
    for alpha in alphas:
        for beta in betas:
            plt.plot(double_exponential_smoothing(series, alpha, beta), label="Alpha {}, beta {}".format(alpha, beta))
    plt.plot(series.values, label = "Actual")
    plt.legend(loc="best")
    plt.axis('tight')
    plt.title("Double Exponential Smoothing")
    plt.grid(True)
    
plot_double_exponential_smoothing(data_date_indexed.EnergyConsumption, alphas=[0.9, 0.02], betas=[0.9, 0.02])



# Modeling

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), syle='bmh'):
    
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style='bmh'):
        fig = plt.figure(figsize=figsize)
        layout = (2,2)
        ts_ax = plt.subplot2grid(layout, (0,0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1,0))
        pacf_ax = plt.subplot2grid(layout, (1,1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title('Time Series Analysis Plots\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()
        
tsplot(data_date_indexed.EnergyConsumption, lags=30)

# Take the first difference to remove to make the process stationary
#data_diff = data_date_indexed.EnergyConsumption - data_date_indexed.EnergyConsumption.shift(1)

#tsplot(data_diff[1:], lags=30)

In [7]:
#Sarima

#Set initial values and some bounds
ps = range(0, 5)
d = 1
qs = range(0, 5)
Ps = range(0, 5)
D = 1
Qs = range(0, 5)
s = 5

#Create a list with all possible combinations of parameters
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

# Train many SARIMA models to find the best set of parameters
def optimize_SARIMA(parameters_list, d, D, s):
    """
        Return dataframe with parameters and corresponding AIC
        
        parameters_list - list with (p, q, P, Q) tuples
        d - integration order
        D - seasonal integration order
        s - length of season
    """
    
    results = []
    best_aic = float('inf')
    
    for param in tqdm(parameters_list):
        try: 
            model = sm.tsa.statespace.SARIMAX(data_date_indexed.EnergyConsumption, order=(param[0], d, param[1]),
                                               seasonal_order=(param[2], D, param[3], s)).fit(disp=-1)
        except:
            continue
            
        aic = model.aic
        
        #Save best model, AIC and parameters
        if aic < best_aic:
            best_model = model
            best_aic = aic
            best_param = param
        results.append([param, model.aic])
        
    result_table = pd.DataFrame(results)
    result_table.columns = ['parameters', 'aic']
    #Sort in ascending order, lower AIC is better
    result_table = result_table.sort_values(by='aic', ascending=True).reset_index(drop=True)
    
    return result_table

result_table = optimize_SARIMA(parameters_list, d, D, s)

#Set parameters that give the lowest AIC (Akaike Information Criteria)
p, q, P, Q = result_table.parameters[0]

best_model = sm.tsa.statespace.SARIMAX(data_date_indexed.EnergyConsumption, order=(p, d, q),
                                       seasonal_order=(P, D, Q, s)).fit(disp=-1)

print(best_model.summary())

NameError: name 'product' is not defined

In [8]:
# save the output of the summary of the model into a separate file
with open("sarima_summary.txt", "w") as summary1:
    summary1.write(str(best_model.summary()))


NameError: name 'best_model' is not defined

In [9]:
### Long short-term Memory (LSTM)

#This is a deep learning model that can handle time seris data with long-term dependencies

In [ ]:



# Check for missing or duplicated timestamps
missing_timestamps = data['Timestamp'].isnull().sum()
duplicated_timestamps = data['Timestamp'].duplicated().sum()

print(f"Missing timestamps: {missing_timestamps}")
print(f"Duplicated timestamps: {duplicated_timestamps}")

# Ensure timestamps are properly formatted and consistent
try:
    data['Timestamp'] = pd.to_datetime(data['Timestamp'])
    print("Timestamps are properly formatted.")
except Exception as e:
    print(f"Error in timestamp formatting: {e}")
